# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **  This is a binary classification problem.  We want partition the student vector space into students who need intervention and students who do not.  Since we have a discrete number of outcomes (2), this is clearly a classification problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.keys()) - 1

# TODO: Calculate passing students
n_passed = sum(student_data['passed'] == 'yes')

# TODO: Calculate failing students
n_failed = n_students - n_passed

# TODO: Calculate graduation rate
grad_rate = n_passed / float(n_students) * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [11]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test/float(len(X_all)), random_state=4)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

1. Support Vector Machines (SVM)
    * __Works well when the feature count is high__
        * Our dataset has a high feature count -- 48 features vs. only ~400 points)
    * **Does not work well when the dataset is really large**
        * **This dataset is fairly small so this downside of SVMs is not a big issue.**
    * SVMs using complex transforms can make it hard to interpret the results and/or understand why the algorithm is performing poorly (or well).
    * A list of SVM use-cases can be found [here](http://www.clopinet.com/SVM.applications.html).  It appears to be popular in some geophysical applications, but there are also references in there that cite it for facial recognition studies, etc.
2. K Nearest Neighbors (KNN)
    * __Works well with highly irregular boundaries.__
        * Since our feature space is large, it's hard to visualize whether we have irregular boundaries or not, but in the event that the decision boundaries in our dataset are very complex, it seems like a good idea to have a model that would perform well in those circumstances.
    * __Should work well for imbalanced data.__
        * Since the algorithm is only taking into account the local neighbors, it doesn't care about the global label distribution.
    * __Very fast training stage__
        * Our cost conscious board will probably like that.  Admittedly, we pay for the fast training with slower prediction...
    * Generally takes longer for predictions than other model types (particularly on really large datasets).
        * __Our dataset is small, so this shouldn't be a big concern.__
    * Very simple so interpreting the model results is straight forward.
    * Apparently this model has been used to help diagnose [breast cancer](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2243774/) and [heart disease](https://arxiv.org/pdf/1508.02061.pdf).
3. AdaBoost
    * **Tends to avoid overfitting** unless the underlying learners are overfitting
    * AdaBoost (with decision trees as the weak learners) is often referred to as the best out-of-the-box classifier<sup>1</sup>.
        - **It frequently has a smaller parameter space to tune (as long as simple weak-learners are chosen).**
        - **Decision trees tend to work well for imbalanced data** (and our data is slightly imbalanced).
    * Is [sensitive to noisy data and outliers](http://www.nickgillian.com/wiki/pmwiki.php/GRT/AdaBoost#Disadvantages).  Presumably this is because it gives preferential treatment to points that it has a hard time classifying.
    * According to the [course materials](https://www.udacity.com/api/nodes/5446988865/supplemental_media/intro-to-boostingpdf/download), AdaBoost generally does better on larger datasets.  We have a small dataset, so that might be a good reason to avoid it.  However it's simplicity for tuning and tendency to avoid overfitting make it a good thing to at least try.
    * Apparently AdaBoost is very successful at [detecting faces in images](https://en.wikipedia.org/wiki/Viola%E2%80%93Jones_object_detection_framework).
    
<sup><sup>1</sup>Source: [Wikipedia](https://en.wikipedia.org/wiki/AdaBoost)</sup>

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [12]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [13]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


# TODO: Initialize the three models
clf_A = SVC(random_state=1)
clf_B = KNeighborsClassifier()
clf_C = AdaBoostClassifier(random_state=1)

# Additional models -- Just for fun.
clf_D = GaussianNB()
clf_E = SGDClassifier()
clf_F = GradientBoostingClassifier()
clf_G = DecisionTreeClassifier()

# TODO: Set up the training set sizes
X_train_100 = 100
y_train_100 = 100

X_train_200 = 200
y_train_200 = 200

X_train_300 = 300
y_train_300 = 300

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
for clf in (clf_A, clf_B, clf_C, clf_D, clf_E, clf_F, clf_G):
    for n_x_train, n_y_train in [(X_train_100, y_train_100), (X_train_200, y_train_200), (X_train_300, y_train_300)]:
        _X_train = X_all[:n_x_train]
        _y_train = y_all[:n_y_train]
        _X_test = X_all[n_x_train:]
        _y_test = y_all[n_y_train:]
        train_predict(clf, _X_train, _y_train, _X_test, _y_test)
        print
    print

Training a SVC using a training set size of 100. . .
Trained model in 0.0015 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 0.8772.
Made predictions in 0.0017 seconds.
F1 score for test set: 0.7792.

Training a SVC using a training set size of 200. . .
Trained model in 0.0028 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.8474.
Made predictions in 0.0019 seconds.
F1 score for test set: 0.7987.

Training a SVC using a training set size of 300. . .
Trained model in 0.0053 seconds
Made predictions in 0.0039 seconds.
F1 score for training set: 0.8686.
Made predictions in 0.0014 seconds.
F1 score for test set: 0.7571.


Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0004 seconds
Made predictions in 0.0018 seconds.
F1 score for training set: 0.8750.
Made predictions in 0.0026 seconds.
F1 score for test set: 0.7126.

Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - SVMClassifier**  

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        .0013            |       .0015s           |     .8772        |     .7792       |
| 200               |        .0029            |       .0020s           |     .8474        |     .7987       |
| 300               |        .0061            |       .0014s           |     .8686        |     .7571       |

** Classifer 2 - KNearestClassifier**  

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      .0004s             |    .0027s              |       .8750      |     .7126       |
| 200               |      .0005s             |    .0027s              |       .8600      |     .7407       |
| 300               |      .0006s             |    .0026s              |       .8597      |     .7597       |

** Classifer 3 - AdaBoostClassifier**  

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |            .1255s       |        .0050s          |     .9867        |      .6319      |
| 200               |            .1349s       |        .0045s          |     .8920        |      .6773      |
| 300               |            .1324s       |        .0037s          |     .8585        |      .6726      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
There doesn't seem to be a clear winner in the three models that we choose to for our initial experiment.  In all cases, training and prediction is fast on a dataset of this size.  AdaBoost seems to be the slowest of the algorithms and it's test scores aren't better than the other algorithms.  Assuming that out of the box parameterization is indicative of it's final performance relative to the other algorithms, we can probably remove it from consideration <sup>1</sup>.

With the parameterization that is provided by sklearn out of the box, it seems that Support Vector Machines have slightly better performance (better test scores) than the other models.  SVMs also require less training data to start getting good results which means that more data can be withheld for validation purposes.  Finally, the discrepancy between training and testing scores for SVMs is the smallest so it seems that it is doing the best job of avoiding overfitting.

<sup><sup>1</sup>This seems like a horrible assumption to me, but it seems like that is the direction that this excercise is taking me so for the sake of this exercise, we'll go with it.</sup>

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **
Support Vector Machines take the data that is given to them and they draw a straight line (hyperplane in higher dimensions) that separates the example data into classes.  Obviously it tries to make sure that the labels on each side of the hyperplane have the same labels.  It also tries to draw the line in such a way that it maximizes the distance to any of the points (it maximizes the "margin").  i.e. (from Wikipedia):

<p><a href="https://commons.wikimedia.org/wiki/File:Kernel_Machine.svg#/media/File:Kernel_Machine.svg"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Kernel_Machine.svg/1200px-Kernel_Machine.svg.png" alt="Kernel Machine.svg"></a><br>By <a href="//commons.wikimedia.org/w/index.php?title=User:Alisneaky&amp;action=edit&amp;redlink=1" class="new" title="User:Alisneaky (page does not exist)">Alisneaky</a>, svg version by <a href="//commons.wikimedia.org/wiki/User:Zirguezi" title="User:Zirguezi">User:Zirguezi</a> - <span class="int-own-work" lang="en">Own work</span>, <a href="http://creativecommons.org/licenses/by-sa/4.0" title="Creative Commons Attribution-Share Alike 4.0">CC BY-SA 4.0</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=47868867">Link</a></p>

Notice that the lines are drawn such that they have an equal space between the points on either side -- This is maximizin the margin.

Now you might notice that the left image isn't a straight line.  SVM's use something called a "kernal trick" to map low dimesional data into higher dimensions where a hyperplane _can_ be drawn.  As an example, consider a classification problem where you want to know if a user's dart throw hit a dart board.  I can give you the data as "distance to the left of the bullseye, and distance above the bullseye" (i.e. I can give you coordinates in a cartesian plane), or I can give you coordinates on a polar plane.  In the latter case, at some distance (`R_sep`) from the bullseye, the labels will all of a sudden switch from "hit the board" to "missed the board".  The Kernal Trick allows us to re-write the input cartsian coordinates (X, Y) into (X, Y, R).  Once we've added R to the dataset, SVM can find the hyperplane that picks out `R_sep` and Bob's our uncle.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [14]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.preprocessing import StandardScaler

c_opts = [.001, .01, .1, 1, 3, 5, 7, 10]
gamma_opts = ['auto', .1, .01, .001, .0001, .00001]
# TODO: Create the parameters list you wish to tune
parameters = [
    {'kernel': ['rbf', 'sigmoid'], 'gamma': gamma_opts, 'C': c_opts},
    {'kernel': ['poly'], 'C': c_opts, 'gamma': gamma_opts, 'degree': np.arange(1, 10)},
]

# TODO: Initialize the classifier
clf = SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
# f1_score = 2 * (precision * recall) / (precision + recall)
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer, n_jobs=8)

scalar = StandardScaler()
scalar.fit(X_train)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(scalar.transform(X_train), y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print(clf.get_params())

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, scalar.transform(X_train), y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, scalar.transform(X_test), y_test))

{'kernel': 'sigmoid', 'C': 7, 'verbose': False, 'probability': False, 'degree': 3, 'shrinking': True, 'max_iter': -1, 'decision_function_shape': None, 'random_state': None, 'tol': 0.001, 'cache_size': 200, 'coef0': 0.0, 'gamma': 0.001, 'class_weight': None}
Made predictions in 0.0030 seconds.
Tuned model has a training F1 score of 0.8303.
Made predictions in 0.0010 seconds.
Tuned model has a testing F1 score of 0.7586.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
The final model does pretty well with F1 scores of 0.8303 and 0.7586 for training and testing respectively.  That's approximately the same as the untuned model (0.7571), but possibly slightly less overfitting (the original training F1 scores were 0.8686).  We _did_ have to do a lot of work to find this (training and validating a couple hundred models), but spread out over multiple CPUs, it went pretty quickly and once the model is trained, the prediction is fast so it's worth the up-front cost (at least, I hope the board of directors thinks so...) and it's good to know that we probably won't find much better parameters.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.